This is the code for server. The server would calculate the individual trust score as well as the aggregated global model's trust score

# Importing libraries

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch

import torchsummary
from torchsummary import summary


from create_MNIST_datasets import get_MNIST, plot_samples
from alibi.confidence import TrustScore


In [2]:
import syft as sy

# Test set preparation

In [3]:
testing_set = torch.load('server_t.pth')
train_data = torch.load('server.pth')

In [4]:
class autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  
            nn.Conv2d(16, 8, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2), 
            
            nn.Conv2d(8, 4, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2), 
            
        )
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(4, 8, 3, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 16, 2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 32, 2, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 1, 3, padding=1) 
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = torch.sigmoid(x)  
        return x
model = autoencoder()

In [5]:
model = torch.load('auto_encoder.pth')

The server loaded the saved autoencoder model, which they would use to reduce the dimension of the data.

The server would connect to each client seperately and for each client, in the drop down box the connectivity status would show connected. Also, if the clients are connected - there would be a live status shown by animated star in the clients side. The server would first establish connection and send the initial model parameters to the clients.

# Connection establishment with client0

In [6]:
duet_0 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


/Users/syeda/courses/fl/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/Users/syeda/courses/fl/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


# Connection establishment with client1

In [7]:
duet_1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


# Connection establishment with client2

In [8]:
duet_2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


# Model creation

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.fc = nn.Linear(784, 10)
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = F.softmax(x)
        return x


In [10]:
model_server = torch.load('initial_model.pth')
model_server_0 = deepcopy(model_server)
model_server_1 = deepcopy(model_server)
model_server_2 = deepcopy(model_server)

# Initial model parameters

In [11]:
parameters = torch.nn.utils.parameters_to_vector(model_server.parameters())


# The initial model parameters sent to clients

In [12]:
parameters.send(duet_0)

/Users/syeda/courses/fl/lib/python3.8/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:482.)
  return self._grad


In [13]:
parameters.send(duet_1)

In [14]:
parameters.send(duet_2)

The above part send the initial model parameters to the clients and would wait until they get the model update from the client. After running upto this part we would move to the client code to receive the model from here and run upto end and return back here

# Model update from client0

The parts below is after the model update is achieved from the clients and we would run the code upto end to calculate the trust score

In [15]:
duet_0.store.pandas

,ID,Tags,Description,object_type
0,<UID: 99b57a43fd3049aaab0fe35cd74af523>,[],,<class 'torch.Tensor'>


In [16]:
try:
    duet_0.store[0].get()
except Exception as e:
    print(e)

[2022-12-03T13:17:20.059371-0500][CRITICAL][logger]][47219] You do not have permission to .get() Object with ID: <UID: 99b57a43fd3049aaab0fe35cd74af523>Please submit a request.


You do not have permission to .get() Object with ID: <UID: 99b57a43fd3049aaab0fe35cd74af523>Please submit a request.


In [17]:
duet_0.store[0].request(reason="Please approve,updated parameters are needeed for aggregating")

In [18]:
y0 = duet_0.store[0].get()

In [19]:
torch.nn.utils.vector_to_parameters(y0,model_server_0.parameters())

# Model update from client1

In [20]:
duet_1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 8284af6a527f411fb27cb556de681280>,[],,<class 'torch.Tensor'>


In [21]:
try:
    duet_1.store[0].get()
except Exception as e:
    print(e)

[2022-12-03T13:17:21.642983-0500][CRITICAL][logger]][47219] You do not have permission to .get() Object with ID: <UID: 8284af6a527f411fb27cb556de681280>Please submit a request.


You do not have permission to .get() Object with ID: <UID: 8284af6a527f411fb27cb556de681280>Please submit a request.


In [22]:
duet_1.store[0].request(reason="Please approve,updated parameters are needeed for aggregating")

In [23]:
y1 = duet_1.store[0].get()

# Model update for client 2

In [24]:
duet_2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 433e32e28da340af84c4f9e5bc536a3e>,[],,<class 'torch.Tensor'>


In [25]:
try:
    duet_2.store[0].get()
except Exception as e:
    print(e)

[2022-12-03T13:17:22.957912-0500][CRITICAL][logger]][47219] You do not have permission to .get() Object with ID: <UID: 433e32e28da340af84c4f9e5bc536a3e>Please submit a request.


You do not have permission to .get() Object with ID: <UID: 433e32e28da340af84c4f9e5bc536a3e>Please submit a request.


In [26]:
duet_2.store[0].request(reason="Please approve,updated parameters are needeed for aggregating")

In [27]:
y2 = duet_2.store[0].get()

In [28]:
def trust_score(test_samples, model_update, model):
    ts = TrustScore()
    encoded_tests=[]
    model_pred_test=[]
    x_tests=[]
    y_tests=[]
    for idx, (features,labels) in enumerate(test_samples):
        x_tests.append(features)
        predictions = model.encoder(features)
        encoded_tests.append(predictions)
        pred_test = model_update(features)
        model_pred_test.append(pred_test)
        y_tests.append(labels)
        ts.fit(encoded_tests[0].detach().numpy(), y_tests[0], classes=10) 
        score, closest_class = ts.score(encoded_tests[0].detach().numpy(),model_pred_test[0].detach().numpy(), k=5)
        print(score)
        print(np.average(score))

# Trust score calculation local model 0

In [29]:
torch.nn.utils.vector_to_parameters(y0,model_server_0.parameters())

In [30]:
trust_score_local_model_0 = trust_score(testing_set, model_server_0, model)

/var/folders/s3/nn8rqkp92c3413g9w6xsc9y00000gn/T/ipykernel_47219/152961979.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)
Reshaping data from (100, 4, 3, 3) to (100, 36) so k-d trees can be built.
Reshaping data from (100, 4, 3, 3) to (100, 36) so k-d trees can be queried.


[0.91611646 0.34358722 0.49972999 0.90256775 0.85823481 0.90961676
 0.50339453 0.37680687 1.00311859 0.72421246 0.6138791  0.51216341
 0.59302116 0.55437738 0.86090176 1.13677235 0.51656371 1.0131434
 0.94917909 0.7445584  0.71486194 0.79715564 0.63199327 0.62107411
 0.62870972 0.84896971 0.31840206 0.87186778 0.79566815 0.51689551
 0.52922323 1.20822625 0.54374973 0.70027123 0.78147525 0.74015425
 0.79890176 1.28774936 1.19678849 0.79193382 0.68756491 0.85027316
 1.16309566 0.67288066 0.94561179 0.84477671 0.81789819 0.35920462
 0.54494544 0.87405807 0.85475375 0.74229579 0.35880865 0.42799714
 1.27023576 1.07509349 1.00616037 0.82342658 0.97064903 1.01280357
 0.77225156 0.82672859 0.50626132 0.6298686  0.71414943 0.83816074
 0.75553036 0.99467816 0.44784061 0.75281613 0.96723007 1.06639903
 0.46653034 0.68336037 0.84716958 1.18716772 0.91328876 0.84932318
 0.52837709 0.51696602 0.69569312 0.64205454 0.48111738 0.78666242
 0.98189232 0.84028484 0.5509518  0.70315346 0.58820458 0.63673

# Trust score calculation for local model 1

In [31]:
torch.nn.utils.vector_to_parameters(y1,model_server_1.parameters())

In [32]:
trust_score_local_model_1 = trust_score(testing_set, model_server_1, model)

/var/folders/s3/nn8rqkp92c3413g9w6xsc9y00000gn/T/ipykernel_47219/152961979.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)
Reshaping data from (100, 4, 3, 3) to (100, 36) so k-d trees can be built.
Reshaping data from (100, 4, 3, 3) to (100, 36) so k-d trees can be queried.


[1.27317971 0.54494544 1.16309566 0.68562178 0.85027316 1.27023576
 0.79715564 0.51656371 0.71414943 0.85823481 1.13677235 0.50626132
 0.44784061 0.36904181 0.77957727 1.07444388 0.46653034 0.7445584
 0.86090176 1.00311859 0.79193382 0.51689551 0.75281613 1.20822625
 0.98189232 0.82672859 0.82211305 0.52837709 1.01280357 0.84896971
 0.46335351 0.84033501 0.81789819 0.64205454 1.1699276  0.75553036
 0.55437738 0.5509518  0.53620387 0.73599642 0.6138791  0.42799714
 0.52922323 0.90961676 0.71486194 0.78926586 1.19678849 0.46685644
 0.97064903 0.97863253 0.84932318 0.94917909 0.90256775 0.81182868
 0.91328876 0.67288066 0.84415393 0.59302116 0.63199327 0.51216341
 0.81803727 0.79036833 0.58524508 0.54374973 0.78666242 0.84028484
 0.96723007 0.68756491 0.93638213 0.57301467 0.83816074 0.87186778
 1.10601009 0.78147525 0.58820458 0.88669871 1.04140556 0.84716958
 0.60149001 0.74015425 0.34358722 0.91611646 0.50339453 1.28774936
 0.80506287 0.62107411 0.77225156 1.03626809 0.94561179 0.99467

#  Trust score calculation for local model 2

In [33]:
torch.nn.utils.vector_to_parameters(y2,model_server_2.parameters())

In [34]:
trust_score_local_model_2 = trust_score(testing_set, model_server_2, model)

/var/folders/s3/nn8rqkp92c3413g9w6xsc9y00000gn/T/ipykernel_47219/152961979.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)
Reshaping data from (100, 4, 3, 3) to (100, 36) so k-d trees can be built.
Reshaping data from (100, 4, 3, 3) to (100, 36) so k-d trees can be queried.


[1.1699276  1.00311859 0.84280898 1.43548408 0.77957727 0.85823481
 0.52837709 0.54494544 0.49972999 0.70027123 0.83816074 0.99467816
 0.71486194 0.51696602 0.84896971 0.93638213 0.97863253 0.86090176
 1.19678849 0.72421924 0.77225156 0.35920462 0.82672859 0.63199327
 1.10601009 2.14198609 0.59302116 0.94917909 0.89259268 0.78926586
 2.67465081 0.79193382 1.03626809 0.75553036 0.64205454 0.48111738
 0.6696631  0.51656371 0.44135571 2.39779062 0.62107411 0.6138791
 0.68562178 0.7364054  0.90256775 0.54374973 1.07509349 1.28774936
 0.90961676 0.84415393 0.87186778 0.82342658 0.58820458 0.84028484
 0.46335351 0.79890176 1.27023576 0.46653034 0.57301467 0.91328876
 0.82211305 0.84716958 1.04140556 0.84932318 0.68756491 0.58524508
 0.67288066 0.51216341 0.78666242 0.96723007 0.68058911 0.60167317
 0.78923101 0.81789819 0.6298686  1.02676252 0.91611646 0.63673888
 0.71414943 1.13677235 0.79715564 1.01280357 0.98189232 0.79668535
 1.16309566 0.51689551 0.44784061 1.20822625 0.47129653 0.69569

# Trust score calculation for global model

In [35]:
y = (y1+y2+y0)/3
torch.nn.utils.vector_to_parameters(y,model_server.parameters())

In [36]:
trust_score_global_model = trust_score(testing_set, model_server, model)

/var/folders/s3/nn8rqkp92c3413g9w6xsc9y00000gn/T/ipykernel_47219/152961979.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)
Reshaping data from (100, 4, 3, 3) to (100, 36) so k-d trees can be built.
Reshaping data from (100, 4, 3, 3) to (100, 36) so k-d trees can be queried.


[0.74015425 0.54374973 0.7445584  0.57301467 0.6298686  0.51216341
 0.84716958 0.64205454 0.46335351 1.07444388 0.79715564 0.62107411
 0.94917909 2.67465081 0.70027123 0.91611646 1.13677235 0.48111738
 0.68562178 0.86090176 1.16309566 0.51696602 1.07509349 0.97863253
 0.51656371 0.81789819 0.79668535 0.82342658 1.27317971 0.78923101
 0.79193382 1.18716772 0.58524508 1.01280357 0.52837709 1.19678849
 0.37680687 0.54494544 0.34358722 0.78666242 0.94561179 1.1699276
 0.79890176 0.91328876 0.63673888 0.75553036 0.84932318 0.6138791
 0.84896971 0.90256775 0.50626132 0.89259268 0.84280898 0.82672859
 0.82211305 0.63199327 0.83816074 0.69569312 1.10601009 1.20822625
 0.87186778 0.84028484 0.96723007 0.71486194 0.49972999 1.27023576
 0.98189232 0.46653034 0.6696631  0.50339453 0.44784061 0.51689551
 0.84415393 0.77225156 0.85823481 0.42799714 0.52922323 0.59302116
 0.68336037 0.68058911 0.68756491 0.84033501 0.99467816 0.78926586
 0.71414943 0.90961676 0.5509518  0.55437738 0.93638213 0.779577

None of the clients are malicious